In [1]:
from nltk.util import ngrams
from collections import defaultdict
import random

In [2]:
# Function to read Amharic words from a file
def read_amharic_words_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        amharic_words = file.read().split()
    return amharic_words

In [3]:
# Read Amharic words from a file
file_path = 'amh_wikipedia_2021_30K-sentences.txt'  # Replace with the actual file path
amharic_words = read_amharic_words_from_file(file_path)

In [4]:
# Preprocess source text to suitable training corpus
trigrams_amharic = list(ngrams(amharic_words, 3))  # Generate trigrams

In [5]:
# Train the language model using trigrams
trigram_model_amharic = defaultdict(list)
for i in range(len(amharic_words)-2):
    trigram = (amharic_words[i], amharic_words[i+1])
    trigram_model_amharic[trigram].append(amharic_words[i+2])

In [6]:
# Function to generate a sequence of words based on the trigram model
def generate_sequence(seed_words, model, length=5):
    current_words = seed_words.split()
    for i in range(length):
        if len(current_words) < 2:
            break  # Break if there are not enough words for the trigram
        w1, w2 = current_words[-2], current_words[-1]
        next_word = random.choice(model.get((w1, w2), []))  # Use get() to handle cases where the trigram is not found
        if next_word is not None:
            current_words.append(next_word)
        else:
            break  # Break if no prediction is found
    return ' '.join(current_words)

In [7]:
# Prompt the user to enter a seed phrase
seed_phrase = input("Enter a seed phrase: ")

# Generate a sequence of words based on the user's input and the trigram model from the file
predicted_sequence = generate_sequence(seed_phrase, trigram_model_amharic, length=3)
print(predicted_sequence)

Enter a seed phrase:  ፈጠረና ጹናሚው


ፈጠረና ጹናሚው የፉኩሺማ ኒውክሌር ኃይል
